In [ ]:
# Import required libraries
from pyspark.sql import SparkSession
# Import necessary libraries
from pyspark.sql.functions import col, when, lit

In [ ]:
# Initialize Spark session
spark = SparkSession.builder.appName("ReadProcessedData").getOrCreate()

In [ ]:
# Azure Data Lake paths (using the same approach as in InspectRawData)
storage_account_name = "datalakestoragetask"  # Replace with your storage account name
processed_container = "processed"
analytics_ready_container = "analytics-ready"
storage_key = ""  # Replace with your key or credential method

# Configure Spark to access Azure Data Lake
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", storage_key)

# Define paths for processed and analytics-ready data
paths = {
    "price_processed": f"abfss://{processed_container}@{storage_account_name}.dfs.core.windows.net/price_and_stock/price",
    "stock_processed": f"abfss://{processed_container}@{storage_account_name}.dfs.core.windows.net/price_and_stock/stock",
    "analytics_ready": f"abfss://{analytics_ready_container}@{storage_account_name}.dfs.core.windows.net"
}

In [ ]:
# Load DataFrames
price_df = spark.read.format("parquet").load(paths["price_processed"])
stock_df = spark.read.format("parquet").load(paths["stock_processed"])

In [ ]:
# Step 2: Merge Data Using Composite Key (including manufacturer)
merged_df = price_df.join(
    stock_df,
    on=["manufacturer", "manufacturer_pid", "retailer_pid", "order_unit"],  # Updated Composite Key
    how="inner"
)

# Step 3: Clean and Restructure Merged Data
merged_df = merged_df.select(
    col("manufacturer"),  # Now a single column
    col("price").alias("price_amount"),
    col("price_base").alias("price_base"),
    col("currency").alias("currency"),
    col("tax_class").alias("tax_class"),
    col("saleable").alias("is_saleable"),
    col("quantity"),
    col(" replenishment_time"),
    col("deeplink")
)


In [ ]:
# Display the merged DataFrame
print("\n--- Displaying Merged Data ---")
#merged_df.show(n=20, truncate=False)
price_df.printSchema()
merged_df.limit(30).show(truncate=False)
display(merged_df.limit(10).toPandas())  # Display as table-like format




--- Displaying Merged Data ---
root
 |-- manufacturer: string (nullable = true)
 |-- manufacturer_pid: string (nullable = true)
 |-- retailer_pid: string (nullable = true)
 |-- order_unit: string (nullable = true)
 |-- price: double (nullable = true)
 |-- price_base: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- tax_class: string (nullable = true)
 |-- saleable: string (nullable = true)



---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File <command-513621778872262>, line 6
      4 price_df.printSchema()
      5 #merged_df.limit(30).show(truncate=False)
----> 6 display(merged_df.limit(10).toPandas())

File /databricks/python_shell/dbruntime/display.py:152, in Display.display(self, input, *args, **kwargs)
    150     self.display(self.sparkSession.createDataFrame(input))
    151 elif type(input).__module__ == 'pandas.core.frame' and type(input).__name__ == 'DataFrame':
--> 152     self.display(self.sparkSession.createDataFrame(input))
    153 elif type(input).__module__ in ('databricks.koalas.frame', 'pyspark.pandas.frame') and \
    154         type(input).__name__ == 'DataFrame':
    155     index_col = kwargs.get('index_col')

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.p

###  Write Merged Data to Analytics-Ready Container


In [ ]:
merged_df.write.format("parquet").mode("overwrite").save(price_stock_analytics_path)

print("\n--- Merged Data Written to Analytics-Ready Container ---")